In [2]:
!pip install Pillow
!pip install gdown

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import gdown

In [4]:
tf.debugging.set_log_device_placement(True)

In [5]:
tf.__version__

'2.0.0'

In [6]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
strategy = tf.distribute.MirroredStrategy()

In [8]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [9]:
import pathlib

data_dir = %pwd
data_dir = data_dir + '/combined_data'
data_dir = pathlib.Path(data_dir)
print(data_dir)
test_dir = %pwd
print(test_dir)

/tf/combined_data
/tf


In [10]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

11122


In [11]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "retrained_labels.txt"])
CLASS_NAMES

array(['benign', 'malignant'], dtype='<U9')

In [12]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

IMG_HEIGHT = 256
IMG_WIDTH = 256

In [13]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

import PIL.Image
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

In [14]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

Executing op MatchingFiles in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Greater in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReduceJoin in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Maximum in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AnonymousRandomSeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0


In [15]:
#transform the images broek out by the text files are in converted to a list of images similar to those read in from list_ds
with open('train_list.txt', 'r') as f:
        train_list = []
        for line in f:
            line = line.rstrip('\n')
            line= line.replace('data', 'tf', 1)
            train_list.append(line)        
        #train_list = [tf.strings.split(x, '/')[:-1] for x in train_list]
        
        

with open('test_list.txt', 'r') as f:
        test_list = []
        for line in f:
            line = line.rstrip('\n')
            line= line.replace('data', 'tf', 1)
            test_list.append(line)        
        #test_list = [tf.strings.split(x, '/')[-1] for x in test_list]
        
with open('val_list.txt', 'r') as f:
        val_list = []
        for line in f:
            line = line.rstrip('\n')
            line= line.replace('data', 'tf', 1)
            val_list.append(line)        
        #val_list = [tf.strings.split(x, '/')[-1] for x in val_list]  
print(train_list[0])
print(type(train_list[0]))

/tf/combined_data/benign/SOB_B_A-14-22549CD-40-022.jpg
<class 'str'>


In [16]:
#simple verification that the images taken from the text files are actually in the folder we read the files in from the list_ds
#input
count = 0
for x in list_ds:
    
    #print(train_list[0])
    if (str(x.numpy())[2:-1]) in train_list:
        continue
    elif (str(x.numpy())[2:-1]) in val_list:
        continue
    elif (str(x.numpy())[2:-1]) in test_list:
        continue
    else:
        count += 1
        print(str(x.numpy())[2:-1])
        print(count)

Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0


In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_list)
valid_dataset = tf.data.Dataset.from_tensor_slices(val_list)
test_dataset  = tf.data.Dataset.from_tensor_slices(test_list)

In [18]:
# set the shuflle and steps per epochs based on size
train_size = len(list(train_dataset))
print('Train Picture Size = ' + str(train_size))

valid_size = len(list(valid_dataset))
print('Valid Picture Size = ' + str(valid_size))

test_size = len(list(test_dataset))
print('Test Picture Size =  ' + str(test_size))

print(test_size + train_size + valid_size)

train_shuffle_size = train_size
valid_shuffle_size = valid_size
test_shuffle_size = test_size
STEPS_PER_EPOCH = np.ceil(train_size/BATCH_SIZE)
VALID_STEPS_PER_EPOCH = np.ceil(valid_size/BATCH_SIZE)
TEST_STEPS_PER_EPOCH = np.ceil(test_size/BATCH_SIZE)

Train Picture Size = 8837
Valid Picture Size = 1144
Test Picture Size =  1141
11122


In [19]:
for f in train_dataset.take(5):
  print(f.numpy())

for f in valid_dataset.take(5):
  print(f.numpy())

for f in test_dataset.take(5):
  print(f.numpy())

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
b'/tf/combined_data/benign/SOB_B_A-14-22549CD-40-022.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.204433294712596606213536280693657707372_1-283.jpg'
b'/tf/combined_data/benign/SOB_B_A-14-22549AB-200-010.jpg'
b'/tf/combined_data/benign/SOB_B_TA-14-3411F-40-006.jpg'
b'/tf/combined_data/benign/SOB_B_TA-14-21978AB-40-014.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.216820283213818005421900437772111995033_1-005.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.381280286813484985622860668700998869999_1-179.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.8224011612206957419584270413389091077_1-222.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.316699245610247354328827790532537478140_1-239.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.81224121613615658336040014641017971693_1-239.jpg'
b'/tf/combined_data/benign/1.3.6.1.4.1.9590.100.1.2.19059292331143213210

In [20]:
def get_label(file_path):
  # convert the path to a list of path components
    # use '/' for linux
  parts = tf.strings.split(file_path, '/')
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_labeled_ds = train_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
valid_labeled_ds = valid_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
test_labeled_ds = test_dataset.map(process_path, num_parallel_calls=AUTOTUNE)

#see https://www.tensorflow.org/tutorials/load_data/images for above and cells above taken from there.

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [21]:
#verify images and labels are joined.

for image, label in train_labeled_ds.take(5):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  
    
  inputshape = image.numpy().shape

for image, label in test_labeled_ds.take(5):
  
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  inputshape = image.numpy().shape
  
print(inputshape)

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
Image shape:  (256, 256, 3)
Label:  [ True False]
(256, 256, 3)


In [22]:
print(train_labeled_ds.take(1))
print(test_labeled_ds.take(1))
test2 = test_labeled_ds.take(1)
print(test2)


<TakeDataset shapes: ((256, 256, 3), (2,)), types: (tf.float32, tf.bool)>
<TakeDataset shapes: ((256, 256, 3), (2,)), types: (tf.float32, tf.bool)>
<TakeDataset shapes: ((256, 256, 3), (2,)), types: (tf.float32, tf.bool)>


In [41]:
def image_transformation(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_jpeg_quality(image, min_jpeg_quality=75, max_jpeg_quality=100)
    #image = tf.image.random_brightness(image, max_delta=.2)
    return image, label

def prepare_for_training(ds, shuffle_buffer_size, cache=False):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  ds = ds.map(image_transformation, num_parallel_calls=AUTOTUNE)

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

#the above mostly constructed from TensorFlow tutorial:
#https://www.tensorflow.org/tutorials/load_data/images

In [24]:
test1 = prepare_for_training(train_labeled_ds, train_shuffle_size)



Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


image_batch, label_batch = next(iter(test1))
#image_batch, label_batch = train_ds.take(5)
show_batch(image_batch.numpy(), label_batch.numpy())

In [25]:
import keras
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras import optimizers 
from keras.layers.advanced_activations import LeakyReLU

ModuleNotFoundError: No module named 'keras'

In [45]:
#Building a CNN with 1 convulutional layer, an actication layer, a pooling layer then flatten to go into the fully connected
# layer
with strategy.scope():
    model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(100, kernel_size = (3,3), input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)),
            tf.keras.layers.LeakyReLU(alpha=0.1),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(50, kernel_size = (5,5)),
            tf.keras.layers.LeakyReLU(alpha=0.1),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(25, kernel_size = (5,5)),
            tf.keras.layers.LeakyReLU(alpha=0.1),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(15, kernel_size = (3,3)),
            tf.keras.layers.LeakyReLU(alpha=0.1),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(5, kernel_size = (3,3)),
            tf.keras.layers.LeakyReLU(alpha=0.1),
            tf.keras.layers.MaxPooling2D((2,2), padding='same'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128),
            tf.keras.layers.LeakyReLU(alpha=0.1),
            tf.keras.layers.Dense(2, activation = 'softmax')])
    model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])



Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:

In [27]:
from time import time
print(time())

1575382953.3294265


In [46]:
start = time()
model.fit(
        prepare_for_training(train_labeled_ds, train_shuffle_size),
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs = 200,
        validation_data=prepare_for_training(valid_labeled_ds, valid_size),
        validation_steps= VALID_STEPS_PER_EPOCH)

end = time()

print((end - start)/60) 
    


Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Train for 70.0 steps, validate for 9.0 steps
Epoch 1/200
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job

Epoch 15/200
70/70 [==============================] - 188s 3s/step - loss: 0.4729 - accuracy: 0.7599 - val_loss: 0.4872 - val_accuracy: 0.7578
Epoch 16/200
70/70 [==============================] - 187s 3s/step - loss: 0.4620 - accuracy: 0.7714 - val_loss: 0.4840 - val_accuracy: 0.7717
Epoch 17/200
70/70 [==============================] - 190s 3s/step - loss: 0.4611 - accuracy: 0.7742 - val_loss: 0.5370 - val_accuracy: 0.7370
Epoch 18/200
70/70 [==============================] - 189s 3s/step - loss: 0.4641 - accuracy: 0.7744 - val_loss: 0.4912 - val_accuracy: 0.7622
Epoch 19/200
70/70 [==============================] - 186s 3s/step - loss: 0.4604 - accuracy: 0.7756 - val_loss: 0.4918 - val_accuracy: 0.7700
Epoch 20/200
70/70 [==============================] - 188s 3s/step - loss: 0.4419 - accuracy: 0.7858 - val_loss: 0.4748 - val_accuracy: 0.7795
Epoch 21/200
70/70 [==============================] - 187s 3s/step - loss: 0.4514 - accuracy: 0.7798 - val_loss: 0.4718 - val_accuracy: 0.7717

70/70 [==============================] - 187s 3s/step - loss: 0.3062 - accuracy: 0.8410 - val_loss: 0.3382 - val_accuracy: 0.8194
Epoch 130/200
70/70 [==============================] - 190s 3s/step - loss: 0.3087 - accuracy: 0.8392 - val_loss: 0.3478 - val_accuracy: 0.8229
Epoch 131/200
70/70 [==============================] - 189s 3s/step - loss: 0.2999 - accuracy: 0.8442 - val_loss: 0.3190 - val_accuracy: 0.8229
Epoch 132/200
70/70 [==============================] - 190s 3s/step - loss: 0.3047 - accuracy: 0.8405 - val_loss: 0.3325 - val_accuracy: 0.8290
Epoch 133/200
70/70 [==============================] - 192s 3s/step - loss: 0.2952 - accuracy: 0.8460 - val_loss: 0.3253 - val_accuracy: 0.8229
Epoch 134/200
70/70 [==============================] - 188s 3s/step - loss: 0.2957 - accuracy: 0.8422 - val_loss: 0.3342 - val_accuracy: 0.8273
Epoch 135/200
70/70 [==============================] - 195s 3s/step - loss: 0.3023 - accuracy: 0.8411 - val_loss: 0.3036 - val_accuracy: 0.8290
Epoch 

Epoch 186/200
70/70 [==============================] - 185s 3s/step - loss: 0.2745 - accuracy: 0.8570 - val_loss: 0.3648 - val_accuracy: 0.8212
Epoch 187/200
70/70 [==============================] - 183s 3s/step - loss: 0.2828 - accuracy: 0.8555 - val_loss: 0.3226 - val_accuracy: 0.8368
Epoch 188/200
70/70 [==============================] - 182s 3s/step - loss: 0.2910 - accuracy: 0.8511 - val_loss: 0.3131 - val_accuracy: 0.8299
Epoch 189/200
70/70 [==============================] - 183s 3s/step - loss: 0.2740 - accuracy: 0.8542 - val_loss: 0.3042 - val_accuracy: 0.8333
Epoch 190/200
70/70 [==============================] - 183s 3s/step - loss: 0.2739 - accuracy: 0.8567 - val_loss: 0.3282 - val_accuracy: 0.8255
Epoch 191/200
70/70 [==============================] - 184s 3s/step - loss: 0.2749 - accuracy: 0.8571 - val_loss: 0.3428 - val_accuracy: 0.8116
Epoch 192/200
70/70 [==============================] - 184s 3s/step - loss: 0.2709 - accuracy: 0.8583 - val_loss: 0.3009 - val_accuracy:

In [47]:
score = model.evaluate(prepare_for_training(test_labeled_ds, test_shuffle_size), steps = TEST_STEPS_PER_EPOCH, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op __inference_distributed_function_225459 in device /job:localhost/replica:0/task:0/device:GPU:0
9/9 [==============================] - 41s 5s/step - loss: 0.3044 - accuracy: 0.8438
Test loss: 0.3044019175900353
Test accuracy: 0.84375


In [ ]:
#potential to save model if needed.
model.save(
    '/saved_model2/my6',
    save_format='tf'    
)

References:
https://stackoverflow.com/questions/51125266/how-do-i-split-tensorflow-datasets
see paper for more references:
Baseline CNN:  utlized the basis of many CNN from https://towardsdatascience.com/the-4-convolutional-neural-network-models-that-can-classify-your-fashion-images-9fe7f3e5399d, https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5, tutorials in tensorflow.org such as https://www.tensorflow.org/guide/data, https://www.tensorflow.org/tutorials/load_data/images, https://www.tensorflow.org/guide/distributed_training, and https://www.tensorflow.org/guide/gpu.  Also, used articles from medium.com and towardsdatascience.com..  Also, used articles from medium.com and towardsdatascience.com.
Also, used stackoverflow and other internet searches for understanding, trouble shooting, and general use.  Further context can be provided if needed.